# % German

In [ ]:
import folium
import geopandas as gpd
import pandas as pd
from libpysal import graph

In [ ]:
cd /home/lisa/work/people_places_germany/code

In [ ]:
from clusters import assign_clusters
from spatial_autocorrelation import lisa

In [ ]:
clusters = gpd.read_parquet(
    "/data/cluster_data/clusters_umap_freiburg_100_3_gaussian_euclidean_complete_chebyshev_3.pq"
)

In [ ]:
clusters.explore(column=clusters.index, cmap="tab20")

# Raw Data

In [ ]:
demographics = pd.read_parquet(
    "/data/processed_data/Bevoelkerung100M.parquet"
).reset_index()

In [ ]:
gdf, largest_overlap, data = assign_clusters(demographics, clusters)

In [ ]:
# calculate % german
gdf["STATS", "% German"] = (
    data["NATIONALITY", "Germany"].fillna(0) / data[" INSGESAMT", "Total"]
)
data["STATS", "% German"] = (
    data["NATIONALITY", "Germany"].fillna(0) / data[" INSGESAMT", "Total"]
)

In [ ]:
gdf1 = gdf[["ID", "geometry", "cluster", "STATS"]]
gdf1.columns = gdf1.columns.get_level_values(0)

In [ ]:
gdf2 = gdf[["ID", "NATIONALITY"]]
gdf2.columns = gdf2.columns.get_level_values(1)

In [ ]:
gdf1 = gdf1.merge(gdf2, how="left", on="ID")

In [ ]:
# gdf1.explore("STATS", cmap="coolwarm", prefer_canvas=True)

# Compute Spatial Lag

In [ ]:
gdf1 = gdf1.cx[4150000:4170000, 2760000:2780000]

In [ ]:
gdf1 = gdf1[gdf1.cluster.notna()]
queen = graph.Graph.build_contiguity(gdf1, rook=False)
row_wise_queen = queen.transform("R")

In [ ]:
gdf1.columns

In [ ]:
for i in ["STATS", "Germany", "Abroad"]:
    gdf1.loc[:, i + "_lag"] = row_wise_queen.lag(gdf1[i])

In [ ]:
gdf1

In [ ]:
gdf1.to_parquet(
    "/home/lisa/work/people_places_germany/Notebooks/temp_data/%german.parquet"
)

# Local Spatial Autocorrelation

In [ ]:
mi, gdf_05 = lisa(gdf1, "STATS", 0.05)

In [ ]:
print(f"Moran's I: {mi.I}, p-value: {mi.p_sim}")

In [ ]:
# gdf_05.explore("cluster", prefer_canvas=True, cmap=["#d7191c","#fdae61","#abd9e9","#2c7bb6","lightgrey"])

# Cluster Aggregation

In [ ]:
d = []
for i in range(0, int(data["cluster", "cluster"].max()) + 1):
    d.append(
        {
            "% German": data[data["cluster", "cluster"] == i][
                "STATS", "% German"
            ].mean(),
            "% German count": data[data["cluster", "cluster"] == i][
                "STATS", "% German"
            ].count(),
        }
    )

stats = pd.DataFrame(d)

In [ ]:
stats["label"] = stats.index
stats = clusters.merge(stats, on="label")

In [ ]:
m = stats.explore(
    column="% German", cmap="coolwarm", prefer_canvas=True, name="cluster aggregation"
)

gdf_05.explore(
    "sig_cluster",
    prefer_canvas=True,
    cmap=["#d7191c", "#fdae61", "#abd9e9", "#2c7bb6", "lightgrey"],
    name="local spatial autocorrelation",
    m=m,
)

gdf1.explore(
    "STATS", cmap="coolwarm", prefer_canvas=True, vmin=0.85, name="raw data", m=m
)

# Add a layer control to toggle the layers on and off
folium.LayerControl().add_to(m)

# Display the map
m